In [1]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
import ibm_db
import ibm_db_dbi

In [2]:
#conectando ao BigSql
#COLOCAR CREDENCIAIS
db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID=XXXXXX;PWD=XXXXX;", "", "")
conn = ibm_db_dbi.Connection(db)   

In [3]:
# buscando os dados das arestas / quem se relaciona com quem
query = "select * from PROCERGS_MILENA_VILLAR.RELACIONAMENTO2;"
df = pd.read_sql_query(query, conn)
# alterando os nomes das colunas para letras minúsculas
df.rename(str.lower, axis='columns', inplace = True)
df.head()

,src,tgt,rel,weight,edge_key
0,3,4,irmã,3,F
1,5,4,co-autores,4,C
2,5,4,co-autores,4,C
3,6,7,co-autores,2,C
4,7,6,co-autores,2,C


In [4]:
#informando quais colunas da tabela serão o source, target e demais atributos;
#create_using específica qual é o tipo do grafo
# edge_key diferencia / categoriza o tipo de relação
M = nx.from_pandas_edgelist(df, source = 'src', target = 'tgt', edge_key='edge_key', edge_attr=['rel','weight'], create_using=nx.MultiDiGraph())

In [5]:
# verificando se o grafo está considerando a coluna weight como peso
nx.is_weighted(M)

True

In [6]:
# verificando se importou os edges corretamente
sorted(M.edges(data = True, keys = True))[:5]

[(1, 2, 'C', {'rel': 'co-autores', 'weight': 4}),
 (1, 3, 'F', {'rel': 'mãe', 'weight': 5}),
 (1, 4, 'F', {'rel': 'mãe', 'weight': 5}),
 (1, 10, 'F', {'rel': 'irmão', 'weight': 3}),
 (1, 16, 'F', {'rel': 'irmã', 'weight': 3})]

In [7]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True, directed = False)
nt.from_nx(M)
nt.show("m.html")

In [8]:
#buscando os labels dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
df_node = pd.read_sql_query(query, conn)

In [9]:
df_node.head()

,ID,NOME,SZ,GP,LAT,LONGT,P_ARMA,TIPO_ARMA,TIPO_CRIME,NUM_VITIMAS
0,1,Vivianne,15,1,-29.881725,-51.164042,SIM,2,ASSALTO,5
1,2,Eloi,5,1,-29.881725,-51.164042,SIM,2,NÃO,0
2,3,Jade,5,1,-29.881725,-51.164042,NÃO,0,NÃO,0
3,4,Camila,15,2,-29.894814,-51.159835,SIM,4,SEQUESTRO,3
4,5,Eduardo,5,2,-29.894814,-51.159835,SIM,2,ASSALTO,3


In [10]:
#transformando a tabela em um dicionário para atribuir labels aos IDs dos nós
nodelabel = df_node.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(M, nodelabel, 'title')
#M.nodes(data=True)

In [11]:
#buscando os tamanhos dos nós(vértices)
query = "select ID,SZ from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
a = pd.read_sql_query(query, conn)

In [12]:
#transformando a tabela em um dicionário para atribuir os tamanhos aos IDs dos nós
nodeatrib = a.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(M, nodeatrib, 'size')
#M.nodes(data=True)

In [13]:
#buscando os grupos dos nós(vértices)
query = "select ID,GP from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
g = pd.read_sql_query(query, conn)

In [14]:
#transformando a tabela em um dicionário para atribuir os grupos aos IDs dos nós
nodeatrib = g.set_index("ID").T.to_dict('records')[0]
# atribuindo o atributo label
nx.set_node_attributes(M, nodeatrib, 'group')
#M.nodes(data=True)

In [15]:
#buscando o atributo coordenadas dos nós(vértices)
query = "select * from PROCERGS_MILENA_VILLAR.COORDENADAS;"
at_node = pd.read_sql_query(query, conn)
#at_node.head()

In [16]:
#transformando a tabela em um dicionário
c = at_node.set_index("ID").T.to_dict('list')
#adicionando o atributo coordenada
nx.set_node_attributes(M, c, 'pos')
#M.nodes(data=True)

In [17]:
#buscando TIPO DE CRIME, NUM. VÍTIMAS dos nós(vértices)
query = "select ID,P_ARMA,TIPO_ARMA,TIPO_CRIME,NUM_VITIMAS from PROCERGS_MILENA_VILLAR.ATRIBUTOS;"
g = pd.read_sql_query(query, conn)

In [18]:
#transformando a tabela em um dicionário para atribuir aos IDs dos nós
nodeatrib = g.set_index("ID").T.to_dict()
# atribuindo o atributo 
nx.set_node_attributes(M, nodeatrib)
#M.nodes(data=True)

In [19]:
# plotar o grafo
nt = Network("400px", "900px", notebook = True)
nt.toggle_hide_edges_on_drag(False) # esconde as arestas quando clica com o mouse
#nt.barnes_hut() #tipo de layout
# nt.show_buttons(filter_=['physics'])
nt.from_nx(M)
nt.show("mg.html")

### Calculando distância entre nós

In [20]:
# calculando distância em Km
import geopy.distance
def distancia(nodes, i, j):
    p1 = M.nodes[i]['pos']
    p2 = M.nodes[j]['pos']
    return geopy.distance.distance(p1, p2).km

In [21]:
G = nx.Graph()

In [22]:
# Calculando a distância entre todos os nós, criando arestas entre esses nós e adicionando chave 'D', atribuindo () a distancia entre os nós
for u in M:
    for v in M:
        if (u != v):
            dt = distancia(M.nodes, u, v)
            G.add_edge(u, v, distancia = dt)

In [23]:
nt = Network("400px", "900px", notebook = True)
nt.from_nx(G)
nt.show("g.html")

#### Achar todo os nós em um raio de 2km do nó 1.
#### Mostrar distância desses nós em relação ao nó 1.
#### Se o nó não for 'vizinho' de 1, mostrar caminho custo mínimo até esse nó.

In [24]:
# achar todos os nós que estão em um raio de 2km de um determinado nó (1)
distancia, caminho = nx.single_source_dijkstra(G, source= 1, target=None, cutoff= 2, weight='distancia')
distancia

{1: 0,
 3: 0.0,
 2: 0.0,
 4: 1.5068113311624167,
 5: 1.5068113311624167,
 15: 1.8759250723451013,
 14: 1.8759250723451013,
 10: 1.8759250723451013,
 13: 1.8759250723451013,
 12: 1.8759250723451013,
 11: 1.8759250723451013}

In [25]:
# Separando os nós que são vizinhos 
vizinhos = M.adj[1]
dist_vizinho = {k: v for k, v in distancia.items() if k in vizinhos}
# Separando os nós que não são vizinhos
dist_nao_vizinho = {k: v for k, v in distancia.items() if k not in vizinhos}
nao_vizinhos = list(sorted(dist_nao_vizinho.keys()))

In [26]:
# distância dos nós vizinhos
print(dist_vizinho)

{3: 0.0, 2: 0.0, 4: 1.5068113311624167, 10: 1.8759250723451013}


In [27]:
# distância dos nós não vizinhos
print(dist_nao_vizinho)

{1: 0, 5: 1.5068113311624167, 15: 1.8759250723451013, 14: 1.8759250723451013, 13: 1.8759250723451013, 12: 1.8759250723451013, 11: 1.8759250723451013}


In [28]:
# Caminho mínimo dos nós não vizinhos
for n in nao_vizinhos:
    try:
        ccm = nx.dijkstra_path(M,source = 1, target = n)
        print('Caminho de custo mínimo: ',ccm)
    except nx.NetworkXNoPath:
        print("Caminho não encontrado até ",n) 

Caminho de custo mínimo:  [1]
Caminho não encontrado até  5
Caminho de custo mínimo:  [1, 10, 11]
Caminho de custo mínimo:  [1, 3, 12]
Caminho de custo mínimo:  [1, 3, 13]
Caminho não encontrado até  14
Caminho não encontrado até  15


In [29]:
# todos os caminhos dos nós não vizinhos
for n in nao_vizinhos:
    for path in nx.all_simple_paths(M, source=1, target= n, cutoff=None): # cutoff é o tamanho do caminho
        print(path)

[1, 10, 11]
[1, 16, 10, 11]
[1, 2, 3, 4, 13, 12]
[1, 2, 3, 4, 12]
[1, 2, 3, 13, 12]
[1, 2, 3, 13, 4, 12]
[1, 2, 3, 12]
[1, 3, 4, 13, 12]
[1, 3, 4, 12]
[1, 3, 13, 12]
[1, 3, 13, 4, 12]
[1, 3, 12]
[1, 4, 3, 13, 12]
[1, 4, 3, 12]
[1, 4, 13, 3, 12]
[1, 4, 13, 12]
[1, 4, 12]
[1, 10, 11, 12]
[1, 10, 11, 13, 3, 4, 12]
[1, 10, 11, 13, 3, 12]
[1, 10, 11, 13, 12]
[1, 10, 11, 13, 4, 3, 12]
[1, 10, 11, 13, 4, 12]
[1, 10, 12]
[1, 10, 13, 3, 4, 12]
[1, 10, 13, 3, 12]
[1, 10, 13, 12]
[1, 10, 13, 4, 3, 12]
[1, 10, 13, 4, 12]
[1, 16, 10, 11, 12]
[1, 16, 10, 11, 13, 3, 4, 12]
[1, 16, 10, 11, 13, 3, 12]
[1, 16, 10, 11, 13, 12]
[1, 16, 10, 11, 13, 4, 3, 12]
[1, 16, 10, 11, 13, 4, 12]
[1, 16, 10, 12]
[1, 16, 10, 13, 3, 4, 12]
[1, 16, 10, 13, 3, 12]
[1, 16, 10, 13, 12]
[1, 16, 10, 13, 4, 3, 12]
[1, 16, 10, 13, 4, 12]
[1, 2, 3, 4, 13]
[1, 2, 3, 4, 12, 13]
[1, 2, 3, 13]
[1, 2, 3, 12, 13]
[1, 2, 3, 12, 4, 13]
[1, 3, 4, 13]
[1, 3, 4, 12, 13]
[1, 3, 13]
[1, 3, 12, 13]
[1, 3, 12, 4, 13]
[1, 4, 3, 13]
[1, 4, 3, 12